# Gallery Example: M[2]/M[2]/1 Priority Queue

This example demonstrates a priority queueing system:
- **Arrivals**: Two independent Poisson processes
- **Service**: Exponential service times
- **Classes**: 2 job classes with different priorities
- **Servers**: 1 server
- **Scheduling**: HOL (Head-of-Line) priority

Priority queues model systems where certain customers receive preferential treatment.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_prio(): Network {    """Create M[2]/M[2]/1 priority queueing model"""    model = Network("M[2]/M[2]/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.HOL)  # Head-of-Line priority    sink = Sink(model, "mySink")        # Block 2: classes with priorities    oclass1 = OpenClass(model, "myClass1", 1)  # Priority 1 (highest)    source.setArrival(oclass1, Exp(1))        # λ₁ = 1    queue.setService(oclass1, Exp(4))         # μ₁ = 4        oclass2 = OpenClass(model, "myClass2", 0)  # Priority 0 (lower)    source.setArrival(oclass2, Exp(0.5))      # λ₂ = 0.5    queue.setService(oclass2, Exp(4))         # μ₂ = 4        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass1, source, queue, 1.0)    P.addRoute(oclass1, queue, sink, 1.0)    P.addRoute(oclass2, source, queue, 1.0)    P.addRoute(oclass2, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mm1_prio()println(f"\nClass 1 (High Priority): λ₁=1, μ₁=4, ρ₁=0.25")println(f"Class 2 (Low Priority):  λ₂=0.5, μ₂=4, ρ₂=0.125")println(f"Total system load: ρ = 0.375")

## Priority Queue Analysis

In HOL (Head-of-Line) priority systems:
- **High priority jobs** are served before any low priority jobs
- **Preemption**: Non-preemptive (current job completes before priority takes effect)
- **Waiting time**: High priority jobs experience reduced delays
- **Performance trade-off**: Low priority jobs may experience significantly longer waits

For M/M/1 priority queue with classes i=1,2:
- Class 1 sees effective load ρ₁ only
- Class 2 sees combined load from both classes

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare with equivalent FCFS system
println("\n=== Priority vs FCFS Comparison ===")

fun create_equivalent_fcfs(): Network {
    """Create equivalent FCFS system"""
    val model_fcfs = Network("M[2]/M[2]/1-FCFS")
    val source = Source(model_fcfs, "Source")
    val queue = Queue(model_fcfs, "Queue", SchedStrategy.FCFS)  # FCFS instead of HOL
    val sink = Sink(model_fcfs, "Sink")
// Same classes and rates
    val oclass1 = OpenClass(model_fcfs, "Class1")
    source.setArrival(oclass1, Exp(1))
    queue.setService(oclass1, Exp(4))
    
    val oclass2 = OpenClass(model_fcfs, "Class2")
    source.setArrival(oclass2, Exp(0.5))
    queue.setService(oclass2, Exp(4))
    
    val P = model_fcfs.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, sink, 1.0)
    P.addRoute(oclass2, source, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model_fcfs.link(P)
    
    return model_fcfs

val model_fcfs = create_equivalent_fcfs()
val solver_prio = MVA(model)
val solver_fcfs = MVA(model_fcfs)

val avg_table_prio = solver_prio.avgTable
val avg_table_fcfs = solver_fcfs.avgTable

println("Priority System (HOL):")
println(avg_table_prio)

println("\nFCFS System:")
println(avg_table_fcfs)
// Extract response times (this is approximate - exact structure may vary)
try:
// Attempt to extract metrics - structure may need adjustment
    println("\nPriority Impact Analysis:")
    println("• High priority jobs: Reduced waiting time vs FCFS")
    println("• Low priority jobs: Increased waiting time vs FCFS")
    println("• Overall system: Same total throughput and utilization")
    println("• Trade-off: Performance improvement for one class at expense of another")
except Exception as e:
    println(f"Detailed comparison error: {e}")
    println("Note: Priority scheduling redistributes waiting times between classes")

In [ ]:
// Analyze priority effects under different loads
println("\n=== Priority Effects Under Different Loads ===")

fun create_priority_load_scenario(lambda1, lambda2): Network {
    """Create priority system with specified arrival rates"""
    val model_load = Network(f"Priority-λ1={lambda1}-λ2={lambda2}")
    val source = Source(model_load, "Source")
    val queue = Queue(model_load, "Queue", SchedStrategy.HOL)
    val sink = Sink(model_load, "Sink")
    
    val oclass1 = OpenClass(model_load, "HighPri", 1)
    source.setArrival(oclass1, Exp(lambda1))
    queue.setService(oclass1, Exp(4))
    
    val oclass2 = OpenClass(model_load, "LowPri", 0)
    source.setArrival(oclass2, Exp(lambda2))
    queue.setService(oclass2, Exp(4))
    
    val P = model_load.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, sink, 1.0)
    P.addRoute(oclass2, source, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model_load.link(P)
    
    return model_load
// Test different load scenarios
val load_scenarios = [
    (0.5, 0.5, "Balanced Load"),
    (1.5, 0.5, "High Priority Heavy"),
    (0.5, 1.5, "Low Priority Heavy"),
    (1.0, 1.0, "High Total Load")
]

println("Scenario          | λ₁  | λ₂  | ρ_total | System Status")
println("-" * 60)

for lambda1, lambda2, scenario_name in load_scenarios:
    try:
        val model_scenario = create_priority_load_scenario(lambda1, lambda2)
        val solver_scenario = MVA(model_scenario)
        val avg_table_scenario = solver_scenario.avgTable
        
        val rho_total = (lambda1 + lambda2) / 4  # Total load
        val status = "Stable" if rho_total < 1 else "Unstable"
        
        println(f"{scenario_name:16s}  | {lambda1:3.1f} | {lambda2:3.1f} |  {rho_total:5.3f}  | {status}")
        
    except Exception as e:
        val rho_total = (lambda1 + lambda2) / 4
        status = "Error/Unstable" if rho_total >= 1 else "Error"
        println(f"{scenario_name:16s}  | {lambda1:3.1f} | {lambda2:3.1f} |  {rho_total:5.3f}  | {status}")

println("\nKey Insights:")
println("1. Priority systems have same stability condition as FCFS: ρ_total < 1")
println("2. High priority traffic gets preferential treatment regardless of load mix")
println("3. Low priority performance degrades more severely under high total load")
println("4. Priority scheduling is most beneficial under moderate to high loads")

In [ ]:
// Theoretical priority queue analysis
println("\n=== Theoretical Priority Queue Formulas ===")
// For M/M/1 priority queue with 2 classes
lambda1, lambda2 = 1.0, 0.5
mu1, mu2 = 4.0, 4.0
rho1, rho2 = lambda1/mu1, lambda2/mu2
val rho_total = rho1 + rho2

println(f"System Parameters:")
println(f"λ₁ = {lambda1}, μ₁ = {mu1}, ρ₁ = {rho1}")
println(f"λ₂ = {lambda2}, μ₂ = {mu2}, ρ₂ = {rho2}")
println(f"ρ_total = {rho_total}")

if rho_total < 1:
// Theoretical waiting times for priority queue
// Class 1 (high priority): sees only its own traffic
    val w1_theory = rho1 / (mu1 * (1 - rho1))
// Class 2 (low priority): sees traffic from both classes
    val w2_theory = (rho1 + rho2) / (mu2 * (1 - rho1) * (1 - rho_total))
// Total response times (waiting + service)
    val r1_theory = w1_theory + 1/mu1
    val r2_theory = w2_theory + 1/mu2
    
    println(f"\nTheoretical Results:")
    println(f"Class 1 (High Priority):")
    println(f"  Waiting time: {w1_theory:.4f}")
    println(f"  Response time: {r1_theory:.4f}")
    
    println(f"\nClass 2 (Low Priority):")
    println(f"  Waiting time: {w2_theory:.4f}")
    println(f"  Response time: {r2_theory:.4f}")
    
    println(f"\nPriority Advantage:")
    println(f"  Response time ratio (Class2/Class1): {r2_theory/r1_theory:.2f}")
    println(f"  Class 1 benefits from priority by factor of {r2_theory/r1_theory:.2f}")
// Compare with FCFS theoretical result
// For FCFS M/M/1: W = ρ/(μ(1-ρ)) for combined system
    val mu_eff = (lambda1*mu1 + lambda2*mu2)/(lambda1 + lambda2)  # Effective service rate
    val w_fcfs = rho_total / (mu_eff * (1 - rho_total))
    val r_fcfs = w_fcfs + 1/mu_eff
    
    println(f"\nFCFS Comparison:")
    println(f"  FCFS response time: {r_fcfs:.4f} (same for both classes)")
    println(f"  Priority Class 1 improvement: {r_fcfs/r1_theory:.2f}x better")
    println(f"  Priority Class 2 degradation: {r2_theory/r_fcfs:.2f}x worse")

else:
    println(f"\nSystem is unstable (ρ = {rho_total:.3f} ≥ 1)")
    println(f"Theoretical analysis not applicable for unstable systems.")